В этом ноутбуке я хочу сделать рекомендательную систему для последующей публикации через Streamlit и Heroku

Смысл модели - мы вводим itemid товара, система рекомендует itemid других товаров, которые могут подойти

Чтобы это было более "человечески" нужно сделать баз данных где itemid соответствует название товара.
Тогда получится так - вводим itemid (не более __ цифр), система говорит - Вы ввели (название товара),
рекомендуем посмотреть так же - список рекомендаций тоже с названием товаров


Итак, для начала сделаем базу, в которой каждому itemid будет соответствовать название товара.
Сделать это придется через asin (сделал это предварительно в другом ноутбуке)

In [1]:
# Импортируем библиотеки
import pandas as pd
from tqdm import tqdm
import numpy as np
import nmslib
import pickle

In [57]:
# подтянем базу данных
titles = pd.read_csv(
    '/Users/alexeizh/Alexeizh_DST/Unit_7/final_prjt/titles.csv')

In [59]:
titles.drop(['Unnamed: 0'], axis=1, inplace=True)

In [62]:
titles.head(3)

,itemid,asin,title
0,37138.0,B01CPNIEQG,['Primal Palate Organic Spices Adobo Seasoning...
1,17322.0,B006F63M8U,['Milk Chocolate Swiss Miss Hot Cocoa Mix 60-0...
2,5600.0,B00112O8NG,"['Torani Syrup, Sugar Free Vanilla, 25.4 oz']"


In [64]:
titles_for_system = titles.set_index('itemid')

In [88]:
titles.title = titles.title.apply(
    lambda x: x.replace('[', '').replace(']', ''))

База готова. Теперь перенесем из ноутбука с моделью эмбеддинги и займемся предсказаниями.

In [66]:
with open('embeddings.pickle', 'rb') as f:
    item_embeddings = pickle.load(f)

In [67]:
item_embeddings

array([[-0.01912594, -0.0308455 ,  0.01930645, ..., -0.03762782,
        -0.02805207, -0.00589758],
       [ 0.2525154 ,  0.33672673,  0.02435321, ..., -0.17207813,
        -0.23986737, -0.04260866],
       [ 0.0412655 ,  0.07120393,  0.06147807, ...,  0.00400613,
         0.03309305,  0.01852722],
       ...,
       [-0.02006457,  0.0324237 ,  0.16484843, ...,  0.1166116 ,
         0.04248474,  0.10554128],
       [-0.01973687, -0.03226512, -0.02077842, ..., -0.03738605,
         0.01635616, -0.04132549],
       [-0.005621  ,  0.00618175,  0.00581868, ..., -0.01509469,
         0.02181922,  0.01129925]], dtype=float32)

In [68]:
# import nmslib

# Создаём наш граф для поиска
nms_idx = nmslib.init(method='hnsw', space='cosinesimil')

# Начинаем добавлять наши товары в граф
nms_idx.addDataPointBatch(item_embeddings)
nms_idx.createIndex(print_progress=True)

In [72]:
# Вспомогательная функция для поиска по графу
def nearest_item_nms(itemid, index, n=5):
    nn = index.knnQuery(item_embeddings[itemid], k=n)
    return nn

In [94]:
# Попробуем найти товары, похожие на товар с неким itemid
nbm = nearest_item_nms(58, nms_idx)[0]
nbm

array([   58, 19567, 29327, 11288, 34440], dtype=int32)

In [95]:
# выведем более понятные имена
print(f'Для {titles.title.loc[58]}')
for item in nbm:
    print(f'\n Рекомендуем Вам посмотреть на {titles.title.loc[item]}')

Для 'San Francisco Bay OneCup Decaf French Roast (36 Count) Single Serve Coffee Compatible with Keurig K-cup Brewers Single Serve Coffee Pods, Compatible with Cuisinart, Bunn, iCoffee single serve brewers'

 Рекомендуем Вам посмотреть на 'San Francisco Bay OneCup Decaf French Roast (36 Count) Single Serve Coffee Compatible with Keurig K-cup Brewers Single Serve Coffee Pods, Compatible with Cuisinart, Bunn, iCoffee single serve brewers'

 Рекомендуем Вам посмотреть на 'var aPageStart = (new Date()).getTime();\nvar ue_t0=ue_t0||+new Date();\n\nwindow.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;\nif (window.ue_ihb === 1) {\n\nvar ue_csm = window,\n    ue_hob = +new Date();\n(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!ba){var c=;ba=function(){c.push(c.slice.call(arguments),e.d(),d.ue_id)};ba.replay=function(b){for(var a;a=c.shift();)b(a0,a1,a2)};ba.isStub=1}};e.exec=function(b,a){return f

In [96]:
# Уберем длинные "страшные" названия (какая-то ошибка в изначальном файле json)
titles.title = titles.title.apply(
    lambda x: 'Unamed item' if 'var' in str(x) else x)

In [ ]:
titles.to_csv('titles_new.csv', index=False)

In [ ]:
# Теперь перенесем это в отдельный файл для прототипирования и последующего деплоя на Heroku